<a href="https://colab.research.google.com/github/nihalmascar/OCRusingDeepLearning/blob/main/OCRwithinBoudningBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performing OCR using Deep Learning

## Importing and installing libraries & storage of paths and creation of folder

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q ultralytics
!pip install -q EasyOCR
!pip3 install -q torch torchvision torchaudio
!unzip -q /content/drive/MyDrive/my_model.zip -d /content/model
from ultralytics import YOLO
import os
import cv2
import easyocr

### ALL PATHS

In [3]:
model_path = "/content/model/my_model.pt"
test_image_path = "/content/drive/MyDrive/Vertical.jpg"
predicted_image = '/content/runs/detect/predict/Vertical.jpg'
text_file_path = '/content/runs/detect/predict/labels/Vertical.txt'

## Object Detection

### Performing YOLO algorithm to detect the image

In [33]:
model = YOLO(model_path)
results = model.predict(source =  test_image_path, save=True, save_txt=True, show_labels=False, show_conf=False)
for result in results:
  classes = result.names


image 1/1 /content/drive/MyDrive/Vertical.jpg: 448x640 1 Date of birth, 1 Expiry Date, 1 ID Number, 1 Issue Date, 1 Name, 1 Nationality, 17.5ms
Speed: 2.7ms preprocess, 17.5ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels


### Function definition to extract bounding box

In [5]:
def boundingbox(img_width,img_height,x_center,y_center,width,height):
  #Calculate the coordinates of the bounding box
  x_center_pixel = x_center * img_width
  y_center_pixel = y_center * img_height
  half_width = width * img_width / 2
  half_height = height * img_height / 2

  #Calculate the coordinates of the bounding box
  x_min = int(x_center_pixel - half_width)
  y_min = int(y_center_pixel - half_height)
  x_max = int(x_center_pixel + half_width)
  y_max = int(y_center_pixel + half_height)

  return x_min, y_min, x_max, y_max

### Performing OCR on bounding box data



In [49]:
image = cv2.imread(predicted_image)
detection = []

with open(text_file_path, 'r') as file:
    lines = file.readlines()

for i , line in enumerate(lines):
  parts = line.split()
  if len(parts) == 5:
    class_id, x_center, y_center, width, height = map(float, parts)

    img_width = image.shape[1]
    img_height = image.shape[0]
    x_min, y_min, x_max, y_max = boundingbox(img_width,img_height,x_center,y_center,width,height)

    image = cv2.resize(image, (img_width,img_height))
    cropped_region = image[y_min:y_max, x_min:x_max]
    reader = easyocr.Reader(['en'])
    detection +=  [reader.readtext(cropped_region)]
    detection [i] += [classes[class_id]]


### Final Output

In [61]:
for ((box,name,prob),text) in detection:
  print(f"{text} = {name}")

Expiry Date = 23/03/2025
Name = Nihal Nishanth Walter Mascarenhas
Date of birth = 16/04/2003
Issue Date = 24/03/2023
ID Number = 784-2003-2021616-9
Nationality = India
